In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
        SparkSession.builder
        .appName("TTest_task_1")
        .getOrCreate()
        )

file_path = ("/home/ilya/github/"
             "DE_course_repo/python_projects/"
             "PySpark/data/weather_data.csv")

weather_df = spark.read.csv(file_path,
                            header=True,
                            inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 16:49:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/16 16:49:30 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
weather_temp = weather_df.select(F.mean("temperature")).first()[0]
weather_df = weather_df.fillna(weather_temp)

weather_df.show(10)
weather_df.printSchema()

+----------+----------+------------------+------------------+-------------------+
|station_id|      date|       temperature|     precipitation|         wind_speed|
+----------+----------+------------------+------------------+-------------------+
| station_7|2022-06-28|-6.751842212861652| 23.67004407474563|  5.458999894629757|
| station_4|2020-04-07| -9.57484426026233|2.9858244485444665|  6.828505392085966|
| station_8|2018-12-22| 19.34342035369938| 33.58211407730149|  8.975576079892296|
| station_5|2021-09-09|30.880953114964086|29.110437988411558|  18.26465360842913|
| station_7|2023-02-07|23.459048180599673|49.510444679381074|  3.787868357696922|
|station_10|2018-05-07| 27.52561802563259|11.957938889112151|0.10053105219606229|
| station_3|2019-08-27| 4.287905745111988|26.536867764341793|  1.201928207563403|
| station_7|2023-04-13|19.244547722319886|32.547866755903634| 18.095326833602357|
| station_8|2020-10-10|1.2541744266864718| 32.22955873711718| 12.950329296867263|
| station_5|2018

In [3]:
hotest_d = weather_df.orderBy(F.desc("temperature")).limit(5)
hotest_d.select("date", "temperature").show()

+----------+------------------+
|      date|       temperature|
+----------+------------------+
|2021-08-20|39.982828249354846|
|2023-12-02| 39.96797489293784|
|2022-03-28|  39.8246894248997|
|2019-02-11| 39.76737697836647|
|2020-06-10| 39.69147838355929|
+----------+------------------+



In [6]:
weather_df = weather_df.withColumn("year", F.year("date"))
weather_df = weather_df.withColumn("month", F.month("date"))
weather_df.show()

+----------+----------+-------------------+------------------+-------------------+----+-----+
|station_id|      date|        temperature|     precipitation|         wind_speed|year|month|
+----------+----------+-------------------+------------------+-------------------+----+-----+
| station_7|2022-06-28| -6.751842212861652| 23.67004407474563|  5.458999894629757|2022|    6|
| station_4|2020-04-07|  -9.57484426026233|2.9858244485444665|  6.828505392085966|2020|    4|
| station_8|2018-12-22|  19.34342035369938| 33.58211407730149|  8.975576079892296|2018|   12|
| station_5|2021-09-09| 30.880953114964086|29.110437988411558|  18.26465360842913|2021|    9|
| station_7|2023-02-07| 23.459048180599673|49.510444679381074|  3.787868357696922|2023|    2|
|station_10|2018-05-07|  27.52561802563259|11.957938889112151|0.10053105219606229|2018|    5|
| station_3|2019-08-27|  4.287905745111988|26.536867764341793|  1.201928207563403|2019|    8|
| station_7|2023-04-13| 19.244547722319886|32.54786675590363

In [ ]:
max_year = weather_df.agg(F.max("year")).first()[0]


max_prec = (weather_df.filter(weather_df["year"] == max_year)
            .groupBy("station_id")
            .agg(F.sum("precipitation").alias("total_precipitation")))
max_prec = max_prec.orderBy(F.desc("total_precipitation")).limit(1)
max_prec.select("station_id", "total_precipitation").show()

+----------+-------------------+
|station_id|total_precipitation|
+----------+-------------------+
| station_5|  642.9302626767898|
+----------+-------------------+



In [14]:
avg_temp = weather_df.groupBy("month").agg(F.avg("temperature").alias("avg_temp")).orderBy(F.asc("month"))
avg_temp.show()

+-----+------------------+
|month|          avg_temp|
+-----+------------------+
|    1|11.356518462550754|
|    2| 9.067229891101926|
|    3| 7.244080205633994|
|    4|12.024529009744693|
|    5| 9.902883346912718|
|    6|13.421092297254138|
|    7|6.1857183016954576|
|    8|  10.9678002814186|
|    9| 9.596744236573942|
|   10|  9.09884344821895|
|   11| 7.265889994697494|
|   12|11.218592100674337|
+-----+------------------+



In [3]:
spark.stop()